<a href="https://colab.research.google.com/github/YousefAtefB/StanceCat-COV19/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting up colab environment and downad word2vec



## Mount google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/StanceCat-COV19

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/StanceCat-COV19


## Download word2vec CBOW model ~ 2.64 GB

In [2]:
# !pip install wget
# import wget
# url = "https://bakrianoo.ewr1.vultrobjects.com/aravec/full_uni_cbow_300_twitter.zip"
# wget.download(url, './Downloads')

## unzip the model

In [3]:
# import zipfile
# with zipfile.ZipFile('./Downloads/full_uni_cbow_300_twitter.zip', 'r') as zip_ref:
#   zip_ref.extractall('./Downloads')

# Importing libraries

In [4]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from tqdm import tqdm
from sklearn.metrics import f1_score
import string

!pip install emoji==1.4.2
import emoji

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Data PreProcessing

In [5]:
train = pd.read_csv('Dataset/train.csv')
text_train, category_train, stance_train = train['text'], train['category'], train['stance']

dev = pd.read_csv('Dataset/dev.csv')
text_dev, category_dev, stance_dev = dev['text'], dev['category'], dev['stance']

In [6]:
train.head(10)

,text,category,stance
0,بيل غيتس يتلقى لقاح #كوفيد19 من غير تصوير الاب...,celebrity,1
1,وزير الصحة لحد اليوم وتحديدا هلأ بمؤتمروا الصح...,info_news,1
2,قولكن رح يكونو اد المسؤولية ب لبنان لما يوصل ...,info_news,1
3,#تركيا.. وزير الصحة فخر الدين قوجة يتلقى أول ج...,celebrity,1
4,وئام وهاب يشتم الدول الخليجية في كل طلة اعلامي...,personal,0
5,"لقاح #كورونا في أميركا.. قلق متزايد من ""التوزي...",info_news,0
6,لبنان اشترى مليونان لقاح امريكي اذا شلنا يلي ع...,info_news,1
7,من عوارض لقاح كورونا<LF>هو تهكير حسابك عتويتر<...,personal,0
8,هناك 1780 مليونيراً في لبنان. ماذا لو فُرضت ال...,unrelated,0
9,دعبول حضرتك منو انت وتطلب من قائد دولة إسلامية...,info_news,1


In [7]:
dev.head(10)

,text,category,stance
0,#مريم_رجوي: <LF>حظر خامنئي المجرم شراء #لقاح_ك...,info_news,1
1,#الصحة:<LF>•تم إعطاء 259.530 جرعة من لقاح #كور...,plan,1
2,#خادم_الحرمين - حفظه الله - يتلقى الجرعة الأول...,celebrity,1
3,#الصحه_العالميه: لقاحات #كورونا آمنة ولا خوف م...,info_news,1
4,"#وزيرة_الصحة ""#هالة_زايد"" تقول إنه يجرى مراجعة...",info_news,1
5,2️⃣ وانتهى الفريق من الدراسات قبل السريرية ونش...,info_news,1
6,عاجل 🔴 <LF>.<LF><LF>.<LF><LF>وزارة الصحة :<LF>...,plan,1
7,#فيديو | السفير الأميركي لدى السعودية بعد تلقي...,info_news,1
8,تصريحات وبس الحكومة مع السيسي علي حسب اللقطة! ...,info_news,0
9,الاتحاد الاوروبي تفاوض لشراء لقاحات الكورونا م...,info_news,1


In [8]:
text_train, category_train, stance_train = np.array(train['text']), np.array(train['category']), np.array(train['stance'])
text_dev, category_dev, stance_dev = np.array(dev['text']), np.array(dev['category']), np.array(dev['stance'])

print(text_train.shape, category_train.shape, stance_train.shape)
print(text_dev.shape, category_dev.shape, stance_dev.shape)

(6988,) (6988,) (6988,)
(1000,) (1000,) (1000,)


In [9]:
# Clean/Normalize Arabic Text 
# taskeel, longation, ... etc
def clean_str(text):
    search = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ى","\\",'\n', '\t','"','?','؟','!']
    replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا","","","","ي","",' ', ' ',' ',' ? ',' ؟ ',' ! ']
    
    #remove tashkeel
    p_tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(p_tashkeel,"", text)
    
    #remove longation
    p_longation = re.compile(r'(.)\1+')
    subst = r"\1\1"
    text = re.sub(p_longation, subst, text)
    
    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')
    
    for i in range(0, len(search)):
        text = text.replace(search[i], replace[i])
    
    #trim    
    text = text.strip()

    return text

def PreProcessing(text, unk='<UNK>'):

  for i in range(len(text)):

    # remove links
    text[i] = re.sub(r'https?:\/\/.*[\r\n]*', '', text[i], flags=re.MULTILINE)

    # limit repeated emojis to 2
    prev_emoji = None
    cnt = 0
    remove_duplicate_emoji = []
    for c in text[i]:
        if c in emoji.UNICODE_EMOJI['en']:
            if prev_emoji == c and cnt >=2:
                continue
            if prev_emoji != c:
              cnt = 0
            prev_emoji = c
            cnt+=1
        remove_duplicate_emoji.append(c)
    text[i] = "".join(remove_duplicate_emoji)

    # seperate emojis
    text[i] = ''.join((' '+c+' ') if c in emoji.UNICODE_EMOJI['en'] else c for c in text[i])

    # Clean/Normalize Arabic Text
    text[i] = clean_str(text[i]) 

    # remove english words
    text[i] = re.sub(r'\s*[A-Za-z]+\b', '' , text[i])

    # remove numbers
    text[i] = re.sub("\d+", " ", text[i])

    # remove punctuations
    text[i] = text[i].translate(str.maketrans('', '', string.punctuation))

    # tokenize
    text[i] = nltk.tokenize.word_tokenize(text[i])

    # remove stop-words
    stopwords = set(nltk.corpus.stopwords.words("arabic"))
    text[i] = [word for word in text[i] if word not in stopwords]

    # remove emojis
    text[i] = [word for word in text[i] if word not in emoji.UNICODE_EMOJI['en']]

    # remove empty words
    text[i] = [word for word in text[i] if len(word)>2]

    # if the tweet became empty just add add <UNK> token 
    if len(text[i])==0:
      text[i] = [unk]

  return text

In [11]:
pre_text_train = PreProcessing(np.array(text_train))
pre_text_dev = PreProcessing(np.array(text_dev))
print(text_train[0], '\n', pre_text_train[0])
print(text_dev[0], '\n', pre_text_dev[0])
print(len(max(pre_text_train,key=len)))
for word in pre_text_train[0]:
  print(word)

بيل غيتس يتلقى لقاح #كوفيد19 من غير تصوير الابرة و لا السيرنجة و لا الدواء و لابس بولو صيفي في عز الشتاء و يقول ان إحدى مزايا عمر ال 65 عامًا هي انه مؤهل للحصول على اللقاح ... يعنى ما كان يحتاج اللقاح لو كان عمره اصغر من 65 🤔 https://t.co/QQKFFUNwBn 
 ['بيل', 'غيتس', 'يتلقي', 'لقاح', 'كوفيد', 'تصوير', 'الابره', 'السيرنجه', 'الدواء', 'ولابس', 'بولو', 'صيفي', 'الشتاء', 'ويقول', 'احدي', 'مزايا', 'عمر', 'عاما', 'انه', 'مؤهل', 'للحصول', 'علي', 'اللقاح', 'يعني', 'يحتاج', 'اللقاح', 'عمره', 'اصغر']
#مريم_رجوي: <LF>حظر خامنئي المجرم شراء #لقاح_كورونا يعد مجزرة متعمدة بحق الشعب الإيراني<LF><LF>نقل موقع مريم رجوي موقف رئيسة الجمهورية المنتخبة للمقاومة الإيرانية من تصريحات خامنئي المجرم حول حظر استيراد لقاح كورونا من الولايات المتحدة و بريطانيا و فرنسا. <LF>#اللقاح_حق_للناس https://t.co/AYXMbXjVKn 
 ['مريم', 'رجوي', 'حظر', 'خامنئي', 'المجرم', 'شراء', 'لقاح', 'كورونا', 'يعد', 'مجزره', 'متعمده', 'بحق', 'الشعب', 'الايرانينقل', 'موقع', 'مريم', 'رجوي', 'موقف', 'رئيسه', 'الجمهوريه', 'المنتخبه', 'للمقاوم

### Saving preprocessed data

In [13]:
with open('our_processed_train.txt','w', encoding='utf8') as f:
	for i in pre_text_train:
		f.write('%s\n'%i)
print(pre_text_train[0])

['بيل', 'غيتس', 'يتلقي', 'لقاح', 'كوفيد', 'تصوير', 'الابره', 'السيرنجه', 'الدواء', 'ولابس', 'بولو', 'صيفي', 'الشتاء', 'ويقول', 'احدي', 'مزايا', 'عمر', 'عاما', 'انه', 'مؤهل', 'للحصول', 'علي', 'اللقاح', 'يعني', 'يحتاج', 'اللقاح', 'عمره', 'اصغر']


### Loading aravec cbow using gensim

In [14]:
!pip install gensim nltk
import gensim
from gensim.models import word2vec

t_model = gensim.models.Word2Vec.load('./Downloads/full_uni_cbow_300_twitter.mdl')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Building vocabulary

In [16]:
def BuildVocab(pad='<PAD>', unk='<UNK>'):

    vocab = [word for word in t_model.wv.index2word]
    vocab = [pad, unk] + vocab

    id2word = {i: word for i, word in enumerate(vocab)}
    word2id = {word: i for i, word in id2word.items()}
    vocab_size = len(vocab)

    return vocab_size, vocab, id2word, word2id    

In [17]:
vocab_size, vocab, id2word, word2id = BuildVocab()
print(vocab_size)

1259758


### Building embedding matrix to finetune later

In [18]:
def BuildingEmbeddingMatrix():
  matrix_len = vocab_size
  emb_dim = 300

  emb_matrix = np.zeros((matrix_len, emb_dim))

  for i, word in enumerate(vocab):
      try: 
          emb_matrix[i] = t_model.wv[word]
      except KeyError:
          emb_matrix[i] = np.random.normal(scale=0.6, size=(emb_dim, ))
  
  return emb_matrix

In [19]:
emb_matrix = BuildingEmbeddingMatrix()
emb_matrix = torch.from_numpy(emb_matrix)

In [20]:
import gc
del t_model
gc.collect()

0

In [21]:
categories = set(category_train)
print(categories)
category2id = {word:i for i, word in enumerate(list(categories))}
print(category2id['celebrity'])

{'celebrity', 'personal', 'unrelated', 'requests', 'others', 'rumors', 'plan', 'restrictions', 'advice', 'info_news'}
0


# Model Building

### Ideas to try
1) bi-directional
2) pre-training
3) multi-layers
4) BERT
5) transformers notebook
6) packed_padded_sequences
7) pre-trained embedding

### Building datasets

In [22]:
class Dataset(torch.utils.data.Dataset):

  def __init__(self, x, y, pad='<PAD>', unk='<UNK>', word2id=word2id):

    x = x.copy()

    # src lengths to be used in pack padded
    self.seq_lengths = torch.LongTensor(list(map(len, x)))

    for i in range(len(x)):
      x[i] = [word2id[word] if word in word2id else word2id[unk] for word in x[i]]
    
    self.X = torch.nn.utils.rnn.pad_sequence([torch.tensor(sentence) for sentence in x], batch_first=True, padding_value=word2id[pad])

    print(self.X[0])

    # sort sequeces decreasing in size
    self.seq_lengths, perm_idx = self.seq_lengths.sort(0, descending=True)
    self.X = self.X[perm_idx]

    print(self.X.shape)

    print(min(self.seq_lengths))
    
    self.Y = torch.tensor(y)
    self.len = len(x)
    self.pad = pad

  def __len__(self):
    return self.len

  def __getitem__(self, idx):
    return self.X[idx], self.Y[idx], self.seq_lengths[idx]

In [23]:
stance_train_dataset = Dataset(pre_text_train, stance_train + 1)
category_train_dataset = Dataset(pre_text_train, [category2id[category] for category in category_train])

stance_dev_dataset = Dataset(pre_text_dev, stance_dev + 1)
category_dev_dataset = Dataset(pre_text_dev, [category2id[category] for category in category_dev])

tensor([ 9280, 40161, 16060, 43868,     1,  3557, 32246,     1, 10460, 39927,
        74361, 17413,  2353,  2362,  3345, 25753,   554,  3283,    98, 38562,
        10166,    11, 92234,   141,  1872, 92234,  1624,  4666,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])
torch.Size([6988, 98])
tensor(1)
tensor([ 9280, 40161, 16060, 43868,     1,  3557, 32246,     1, 10460, 39927,
        74361, 17413,  2353,  2362,  3345, 25753,   554,  3283,    98, 38562,
        10166,    11, 92234,   141,  1872, 9

### The LSTM model

In [24]:
def create_emb_layer(emb_matrix, non_trainable=False):
    num_embeddings, embedding_dim = emb_matrix.shape
    emb_layer = nn.Embedding(num_embeddings, embedding_dim)
    emb_layer.load_state_dict({'weight': emb_matrix})
    
    if non_trainable:
        emb_layer.weight.requires_grad = False

    return emb_layer, num_embeddings, embedding_dim

class LSTM(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, output_dim, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        self.output_dim = output_dim

        self.embedding, num_embeddings, embedding_dim = create_emb_layer(emb_matrix, True)
        self.lstm = nn.LSTM(emb_dim, hid_dim, dropout=dropout)

        self.fc_out = nn.Linear(hid_dim, output_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, src, src_len):
        #src = [src len, batch size]
        
        embedded = self.embedding(src)
        embedded = self.dropout(embedded)
        #embedded = [src len, batch size, emb dim]

        packed_embedded =  torch.nn.utils.rnn.pack_padded_sequence(embedded, src_len.cpu().numpy(), batch_first=False)

        outputs, (hidden, cell) = self.lstm(packed_embedded)
        #outputs = [src len, batch size, hid dim]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        #outputs are always from the top hidden layer

        prediction = self.fc_out(hidden)
        #prediction = [1, batch size, output dim]

        prediction = prediction.squeeze(0)
        #prediction = [batch size, output dim]

        return prediction        

### Use GPU if any

In [25]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Training function

In [26]:
def train(model, train_dataset, train_dataloader, optimizer, criterion, clip):
    
    model.train()
        
    total_acc_train = 0.0
    total_loss_train = 0.0

    for train_input, train_label, src_len in tqdm(train_dataloader):

        train_input = train_input.to(device).permute(1, 0)
        # print(train_input.shape)
        train_label = train_label.to(device)

        output = model(train_input, src_len)
        # print(output.shape, train_label.shape)
        
        output_dim = output.shape[-1]
        output = output.view(-1, output_dim)
        train_label = train_label.view(-1)

        batch_loss = criterion(output, train_label)

        total_loss_train += batch_loss
        
        acc = torch.sum(torch.argmax(output, -1) == train_label) 
        total_acc_train += acc

        optimizer.zero_grad()

        batch_loss.backward()

        # torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()

    # calculate loss    
    epoch_loss = total_loss_train / len(train_dataset)
    
    # calculate accuracy
    epoch_acc = total_acc_train / len(train_dataset)

    # calculate f1 score
    train_input, train_label, src_len = train_dataset[:]
    train_input = train_input.to(device).permute(1, 0)
    train_label = train_label.to(device)
    output = model(train_input, src_len)

    output_dim = output.shape[-1]
    output = output.view(-1, output_dim)
    train_label = train_label.view(-1)
    y_true, y_pred = train_label, torch.argmax(output, -1)

    f1_macro = f1_score(y_true.cpu(), y_pred.cpu(), average='macro')

    return epoch_loss, epoch_acc, f1_macro

### Evaluating function

In [27]:
def evaluate(model, test_dataset, test_dataloader, criterion):

  model.eval()

  total_acc_test = 0.0
  total_loss_test = 0.0
  
  with torch.no_grad():

    for test_input, test_label, src_len in tqdm(test_dataloader):

      test_input = test_input.to(device).permute(1, 0)
      test_label = test_label.to(device)


      output = model(test_input, src_len)

      batch_loss = criterion(output.view(-1, model.output_dim), test_label.view(-1))

      total_loss_test += batch_loss

      acc = torch.sum(torch.argmax(output, -1)==test_label)
      total_acc_test += acc

    # calculate loss
    total_loss_test /= len(test_dataset)

    # calculate accuracy
    total_acc_test /= len(test_dataset)

    # calculate f1 score
    test_input, test_label, src_len = test_dataset[:]
    test_input = test_input.to(device).permute(1, 0)
    test_label = test_label.to(device)
    output = model(test_input, src_len)

    output_dim = output.shape[-1]
    output = output.view(-1, output_dim)
    test_label = test_label.view(-1)
    y_true, y_pred = test_label, torch.argmax(output, -1)

    f1_macro = f1_score(y_true.cpu(), y_pred.cpu(), average='macro')
  
  return total_loss_test, total_acc_test, f1_macro

### Training and evaluating the model each epoch

In [28]:
def train_evaluate(model, train_dataset, dev_dataset, model_name, batch_size=512, epochs=20, learning_rate=0.01, clip=1):

  train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)

  dev_dataloader = torch.utils.data.DataLoader(dev_dataset, batch_size=batch_size)

  # criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)
  criterion = nn.CrossEntropyLoss()

  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

  use_cuda = torch.cuda.is_available()
  device = torch.device("cuda" if use_cuda else "cpu")
  
  model = model.to(device)
  criterion = criterion.to(device)

  best_f1_macro = 0

  for epoch_num in range(epochs):

    epoch_loss, epoch_acc, train_f1_macro = train(model, train_dataset, train_dataloader, optimizer, criterion, clip)
    dev_loss, dev_acc, dev_f1_macro = evaluate(model, dev_dataset, dev_dataloader, criterion)

    if dev_f1_macro > best_f1_macro:
      best_f1_macro = dev_f1_macro
      torch.save(model.state_dict(), 'best_'+model_name+'.pt')

    print(f'Train = Epochs: {epoch_num + 1} | Loss: {epoch_loss} | Accuracy: {epoch_acc} | f1_macro : {train_f1_macro}')
    print(f'Dev = Epochs: {epoch_num + 1} | Loss: {dev_loss} | Accuracy: {dev_acc} | f1_macro : {dev_f1_macro}')    

  model.load_state_dict(torch.load('best_'+model_name+'.pt'))

  dev_loss, dev_acc, dev_f1_macro = evaluate(model, dev_dataset, dev_dataloader, criterion)

  print(f'Best Dev = Loss: {dev_loss} | Accuracy: {dev_acc} | f1_macro : {dev_f1_macro}')

### Stance Model

In [29]:
INPUT_DIM = vocab_size
OUTPUT_DIM = 3
EMB_DIM = 300 #256
HID_DIM = 512 #512
DROPOUT = 0.5

emb_matrix.to(device)
stance_model = LSTM(INPUT_DIM, EMB_DIM, HID_DIM, OUTPUT_DIM, DROPOUT).to(device)

def init_weights(m):
    for name, param in m.named_parameters():
      if param.requires_grad:
        nn.init.uniform_(param.data, -0.08, 0.08)
        
stance_model.apply(init_weights)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(stance_model):,} trainable parameters')

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/rnn.py:67: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


The model has 1,668,611 trainable parameters


In [30]:
train_evaluate(stance_model, stance_train_dataset, stance_dev_dataset, 'LSTM_stance_model')

100%|██████████| 2/2 [00:00<00:00, 96.32it/s]


Train = Epochs: 1 | Loss: 0.0015660615172237158 | Accuracy: 0.7246708273887634 | f1_macro : 0.29656651358407354
Dev = Epochs: 1 | Loss: 0.0012819812400266528 | Accuracy: 0.8030000329017639 | f1_macro : 0.2969125531521538


100%|██████████| 2/2 [00:00<00:00, 87.95it/s]


Train = Epochs: 2 | Loss: 0.0012623704969882965 | Accuracy: 0.7920721173286438 | f1_macro : 0.3033833173075721
Dev = Epochs: 2 | Loss: 0.0012719965307042003 | Accuracy: 0.8040000200271606 | f1_macro : 0.2972823072656683


100%|██████████| 2/2 [00:00<00:00, 90.27it/s]


...

Train = Epochs: 19 | Loss: 0.0007830747053958476 | Accuracy: 0.857469916343689 | f1_macro : 0.654371589915396
Dev = Epochs: 19 | Loss: 0.00200795941054821 | Accuracy: 0.7780000567436218 | f1_macro : 0.32826233709767844


100%|██████████| 2/2 [00:00<00:00, 87.32it/s]


Train = Epochs: 20 | Loss: 0.000770277576521039 | Accuracy: 0.8631940484046936 | f1_macro : 0.682276073325314
Dev = Epochs: 20 | Loss: 0.0019924493972212076 | Accuracy: 0.7610000371932983 | f1_macro : 0.3223524669566291


100%|██████████| 2/2 [00:00<00:00, 67.23it/s]

Best Dev = Loss: 0.0017544880975037813 | Accuracy: 0.7710000276565552 | f1_macro : 0.3300097136115384


### Category model

In [31]:
INPUT_DIM = vocab_size
OUTPUT_DIM = 10
EMB_DIM = 300 #256
HID_DIM = 512 #512
DROPOUT = 0.5

category_model = LSTM(INPUT_DIM, EMB_DIM, HID_DIM, OUTPUT_DIM, DROPOUT).to(device)

def init_weights(m):
    for name, param in m.named_parameters():
      if param.requires_grad:
        nn.init.uniform_(param.data, -0.08, 0.08)
        
category_model.apply(init_weights)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(category_model):,} trainable parameters')

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/rnn.py:67: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


The model has 1,672,202 trainable parameters


In [32]:
train_evaluate(category_model, category_train_dataset, category_dev_dataset, 'LSTM_category_model')

100%|██████████| 2/2 [00:00<00:00, 71.35it/s]


Train = Epochs: 1 | Loss: 0.0034703644923865795 | Accuracy: 0.4287349581718445 | f1_macro : 0.07318558712477792
Dev = Epochs: 1 | Loss: 0.002991356421262026 | Accuracy: 0.5440000295639038 | f1_macro : 0.07323958333333333


100%|██████████| 2/2 [00:00<00:00, 83.74it/s]


Train = Epochs: 2 | Loss: 0.0029923003166913986 | Accuracy: 0.518460214138031 | f1_macro : 0.08050333182023076
Dev = Epochs: 2 | Loss: 0.0029726296197623014 | Accuracy: 0.5440000295639038 | f1_macro : 0.07074122236671002


100%|██████████| 2/2 [00:00<00:00, 105.66it/s]


...

Train = Epochs: 19 | Loss: 0.0019327620975673199 | Accuracy: 0.659988522529602 | f1_macro : 0.5182668611981727
Dev = Epochs: 19 | Loss: 0.004227809142321348 | Accuracy: 0.4620000123977661 | f1_macro : 0.0943029239739992


100%|██████████| 2/2 [00:00<00:00, 88.26it/s]


Train = Epochs: 20 | Loss: 0.0018963465699926019 | Accuracy: 0.6631367802619934 | f1_macro : 0.5207172463851457
Dev = Epochs: 20 | Loss: 0.0042904154397547245 | Accuracy: 0.4270000159740448 | f1_macro : 0.08887634324478662


100%|██████████| 2/2 [00:00<00:00, 60.42it/s]

Best Dev = Loss: 0.0040560453198850155 | Accuracy: 0.46300002932548523 | f1_macro : 0.11056037495731733
